In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def create_membership(num_clusters, num_points):
    dm = []
    for _ in range(num_points):
        random_values = np.random.rand(num_clusters).round(2)
        dm.append(random_values)
    dm = np.array(dm)
    for i in range(len(dm)):
        dm[i] = (dm[i] / np.sum(dm[i])).round(2)
    return dm

def create_data(num_points):
    data_points = []
    for i in range(num_points):
        values = []
        for j in range(2):
            value = float(input(f'Enter value x[{i}][{j}]: '))
            values.append(value)
        data_points.append(values)
    data_points = np.array(data_points)
    return data_points

def Centroid(data, clust):
    clust = np.array(clust)
    data = np.array(data)
    Centroid = []
    num_clusters = clust.shape[1]
    for i in range(num_clusters):
        numerator = 0
        denominator = 0
        for j in range(len(clust)):
            membership_value = clust[j][i] ** 2
            numerator += membership_value * data[j]
            denominator += membership_value
        if denominator != 0:
            centroid = numerator / denominator
        else:
            centroid = np.nan
        Centroid.append(centroid)
    return Centroid

def update_membership(data, clust, c):
    clust = np.array(clust)
    data = np.array(data)
    c = np.array(c)
    new_clust = np.zeros_like(clust)
    for i in range(len(data)):
        for j in range(clust.shape[1]):
            dist_cj = np.linalg.norm(data[i] - c[j])
            den = 0
            for k in range(clust.shape[1]):
                dist_ck = np.linalg.norm(data[i] - c[k])
                den += (dist_cj / (dist_ck)) ** 2
            new_clust[i][j] = 1 / den
    return new_clust

def fuzzy_c_means(data, num_clusters, threshold=1e-5, max_iter=100):
    num_points = len(data)
    membership = create_membership(num_clusters, num_points)

    iteration = 0
    while iteration < max_iter:
        centroids = Centroid(data, membership)
        new_membership = update_membership(data, membership, centroids)
        diff = np.linalg.norm(new_membership - membership)
        print(f"Iteration {iteration}, difference = {diff}")
        if diff < threshold:
            print("Convergence reached.")
            break
        membership = new_membership
        iteration += 1
    return membership, centroids, iteration

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


# Code pour les images GRAYSCALE
def init_clusters(k, shape):
    centers = np.random.rand(k, 1) * 255
    return centers

def calc_membership(img, centers, m):
    k = centers.shape[0]
    mem = np.zeros((img.shape[0], img.shape[1], k))

    for i in range(k):
        dists = np.abs(img - centers[i])
        mem[:, :, i] = 1 / (dists ** (2 / (m - 1)))

    mem_sum = np.sum(mem, axis=2)
    mem = mem / mem_sum[:, :, np.newaxis]
    return mem

def update_centers(img, mem, m):
    k = mem.shape[2]
    centers = np.zeros((k, 1))

    for i in range(k):
        num = np.sum((mem[:, :, i] ** m) * img)
        denom = np.sum(mem[:, :, i] ** m)
        centers[i] = num / denom if denom != 0 else 0

    return centers

def fcm_segmentation(img, k, m=2, max_iter=100):
    centers = init_clusters(k, img.shape)

    for _ in range(max_iter):
        mem = calc_membership(img, centers, m)
        centers = update_centers(img, mem, m)

    seg_img = np.argmax(mem, axis=2)
    return seg_img

img = cv2.imread('milky-way.jpg', cv2.IMREAD_GRAYSCALE)
img = img.astype(np.float32)

img = img / 255.0

num_clusters = 3
segmented_img = fcm_segmentation(img, num_clusters)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original')
plt.imshow(img, cmap='gray')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('Segmented')
plt.imshow(segmented_img, cmap='gray')
plt.axis('off')

plt.show()


In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


# code pour images RGB
def initialize_clusters(num_clusters):
    cluster_centers = np.random.rand(num_clusters, 3) * 255
    return cluster_centers

def compute_membership(image, cluster_centers, m):
    height, width, _ = image.shape
    num_clusters = cluster_centers.shape[0]
    membership = np.zeros((height, width, num_clusters))
    for c in range(num_clusters):
        distances = np.linalg.norm(image - cluster_centers[c], axis=2)
        membership[:, :, c] = 1 / (distances ** (2 / (m - 1)) + 1e-6)
    membership_sum = np.sum(membership, axis=2)
    membership = membership / membership_sum[:, :, np.newaxis]
    return membership

def update_cluster_centers(image, membership, m):
    height, width, _ = image.shape
    num_clusters = membership.shape[2]
    cluster_centers = np.zeros((num_clusters, 3))
    for c in range(num_clusters):
        numerator = np.sum((membership[:, :, c] ** m)[:, :, np.newaxis] * image, axis=(0, 1))
        denominator = np.sum(membership[:, :, c] ** m)
        if denominator != 0:
            cluster_centers[c] = numerator / denominator
        else:
            cluster_centers[c] = 0
    return cluster_centers

def fcm_segmentation(image, num_clusters, m=2, max_iter=100):
    cluster_centers = initialize_clusters(num_clusters)
    for _ in range(max_iter):
        membership = compute_membership(image, cluster_centers, m)
        cluster_centers = update_cluster_centers(image, membership, m)
    segmented_image = np.argmax(membership, axis=2)
    return segmented_image

image = cv2.imread('/content/milky-way.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = image.astype(np.float32)
image = image / 255.0
num_clusters = 30
segmented_image = fcm_segmentation(image, num_clusters)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(image)
plt.axis('off')
plt.subplot(1, 2, 2)
plt.title('Segmented Image')
plt.imshow(segmented_image, cmap='viridis')
plt.axis('off')
plt.show()
